# Introduction to Data Science

# The CRISP-DM approach to Data Science Tasks

_________________________

# 1. Business Understanding
                    
### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:  

+ Assess situation:  

+ Determine data mining goals:  

+ Produce project plan:  



In [1]:
import pandas as pd
import basedosdados as bd

# 2. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## 2.a. Collect initial data:  

In [2]:
# Define algumas variáveis úteis
dataset_id = "br_inep_indicadores_educacionais"
billing_project_id="trabalho-amd"

Agora visualizamos a estrutura do dataset

In [3]:
bd.list_dataset_tables(dataset_id=dataset_id)


table_id: 
	brasil 
-----------------------------------------------------------------------------------------------
table_id: 
	brasil_remuneracao_docentes 
-----------------------------------------------------------------------------------------------
table_id: 
	escola 
-----------------------------------------------------------------------------------------------
table_id: 
	escola_nivel_socioeconomico 
-----------------------------------------------------------------------------------------------
table_id: 
	fluxo_educacao_superior 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio_taxas_transicao 
-----------------------------------------------------------------------------------------------
table_id: 
	regiao 
--------------------------------------------------------------------------------------

Fazendo o download de cada uma das tabelas do dataset:

+ brasil

In [4]:
#bd.download(dataset_id=dataset_id, table_id='brasil', savepath='data/brasil.csv', billing_project_id=billing_project_id)
df_brasil = pd.read_csv('data/brasil.csv')

+ brasil_remuneracao_docentes

In [5]:
#bd.download(dataset_id=dataset_id, table_id='brasil_remuneracao_docentes', savepath='data/brasil_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_brasil_remuneracao = pd.read_csv('data/brasil_remuneracao_docentes.csv')

+ escola

In [6]:
#bd.download(dataset_id=dataset_id, table_id='escola', savepath='data/escola.csv', billing_project_id=billing_project_id)
df_escola = pd.read_csv('data/escola.csv')

+ escola_nivel_socioeconomico

In [7]:
#bd.download(dataset_id=dataset_id, table_id='escola_nivel_socioeconomico', savepath='data/escola_nivel_socioeconomico.csv', billing_project_id=billing_project_id)
df_escola_nivel_socioeconomico = pd.read_csv('data/escola_nivel_socioeconomico.csv')

+ municipio

In [8]:
#bd.download(dataset_id=dataset_id, table_id='municipio', savepath='data/municipio.csv', billing_project_id=billing_project_id)
df_minicipio = pd.read_csv('data/municipio.csv')

In [9]:
#bd.download(dataset_id=dataset_id, table_id='municipio_taxas_transicao', savepath='data/municipio_taxas_transicao.csv', billing_project_id=billing_project_id)
df_municipio_taxas = pd.read_csv('data/municipio_taxas_transicao.csv')

In [10]:
#bd.download(dataset_id=dataset_id, table_id='regiao', savepath='data/regiao.csv', billing_project_id=billing_project_id)
df_regiao = pd.read_csv('data/regiao.csv')

In [11]:
#bd.download(dataset_id=dataset_id, table_id='uf', savepath='data/uf.csv', billing_project_id=billing_project_id)
df_uf = pd.read_csv('data/uf.csv')

In [12]:
#bd.download(dataset_id=dataset_id, table_id='uf_remuneracao_docentes', savepath='data/uf_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_uf_remuneracao = pd.read_csv('data/uf_remuneracao_docentes.csv')

## 2.b. Describe Data

Começando pela tabela `escola` que é a mais importante do dataset, podemos ver que ela é formada por 208 colunas, destas 205 numéricas e 3 categóricas.

In [24]:
df_escola.info()
df_escola.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637557 entries, 0 to 2637556
Columns: 208 entries, ano to icg_nivel_complexidade_gestao_escola
dtypes: float64(202), int64(3), object(3)
memory usage: 4.1+ GB


,ano,id_municipio,rede,id_escola,localizacao,atu_ei,atu_ei_creche,atu_ei_pre_escola,atu_ef,atu_ef_anos_iniciais,...,ied_ef_anos_finais_nivel_4,ied_ef_anos_finais_nivel_5,ied_ef_anos_finais_nivel_6,ied_em_nivel_1,ied_em_nivel_2,ied_em_nivel_3,ied_em_nivel_4,ied_em_nivel_5,ied_em_nivel_6,icg_nivel_complexidade_gestao_escola
0,2013,1100015,municipal,11024542,rural,NaN,NaN,NaN,5.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
1,2013,1100015,municipal,11025000,rural,NaN,NaN,NaN,6.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
2,2013,1100015,municipal,11025115,rural,11.0,NaN,11.0,13.0,NaN,...,30.8,7.7,7.7,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
3,2013,1100015,municipal,11047461,urbana,19.3,NaN,19.3,20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
4,2013,1100023,municipal,11007850,rural,NaN,NaN,NaN,18.1,17.2,...,71.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
5,2013,1100023,estadual,11048620,urbana,NaN,NaN,NaN,29.9,27.8,...,33.3,44.4,5.6,0.0,0.0,0.0,46.6,46.7,6.7,nivel 6
6,2013,1100023,municipal,11007931,urbana,26.1,NaN,26.1,27.7,27.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 5
7,2013,1100049,municipal,11026294,rural,13.0,NaN,13.0,17.1,17.6,...,42.9,0.0,14.3,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
8,2013,1100049,estadual,11026014,urbana,NaN,NaN,NaN,19.0,19.8,...,71.4,14.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
9,2013,1100049,estadual,11026545,urbana,NaN,NaN,NaN,27.5,27.4,...,33.3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3


Agora filtrando os dados apenas do Ensino Médio:

In [ ]:
df_brasil

### Explore Data

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

#### Peso vs. Altura

In [ ]:
colors = {'M':'blue', 'F':'red'}
plt.scatter(y = df['Height'], x = df['Weight'], alpha = 0.1, c=df['Sex'].map(colors))
# sns.jointplot(df['Height'], df['Weight'])

plt.ylabel('Altura dos Atletas')
plt.xlabel('Peso dos Atletas')
plt.rcParams.update({'figure.figsize':(10,10), 'figure.dpi':100})
plt.show()

In [ ]:
plt.clf()

## 3. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 